In [2]:
import math
import itertools
import numpy as np
import random
from dataclasses import dataclass
import matplotlib.pyplot as plt
from collections import Counter
import sympy as sp
from sympy import gcd, gcdex
from itertools import product

In [3]:
ncol = 12
nrow = 6
L=12
l_h = L // 2
P=768

In [4]:
def generate_coprime_array():
    candidates = [i for i in range(2, P) if math.gcd(i, P) == 1]
    
    result = random.choices(candidates, k=L)
    return result
# a_vec = np.array(generate_coprime_array())
a_vec = [763, 679, 397, 61, 697, 373, 
         289, 257, 625, 41, 193, 449]

In [5]:
G = np.zeros((36, 12), dtype=int)
for i in range(6):
    for j in range(6):
        G[6*i+j, i] = (1 - a_vec[6+j])
        G[6*i+j, 6+j] = (a_vec[i]-1)

In [6]:
def print_g_matrix(matrix, split_idx=6):
    for row in matrix:
        left = row[:split_idx]
        right = row[split_idx:]
        left_str = " ".join(f"{val:3}" for val in left)
        right_str = " ".join(f"{val:3}" for val in right)
        print(f"{left_str} | {right_str}")

def print_g_smart(matrix, split_idx=6):
    for row in matrix:
        elements = []
        for i, val in enumerate(row):
            if i == split_idx:
                elements.append("|")
            if val == 0:
                elements.append(f"   ·")
            else:
                elements.append(f"{val:4}")
        
        # 行番号を付けて表示
        print(f"{' '.join(elements)}")

In [7]:
G_a = G[[i for i in range(36) if i not in [3, 8]]]
print_g_smart(G_a)

-288    ·    ·    ·    ·    · |  762    ·    ·    ·    ·    ·
-256    ·    ·    ·    ·    · |    ·  762    ·    ·    ·    ·
-624    ·    ·    ·    ·    · |    ·    ·  762    ·    ·    ·
-192    ·    ·    ·    ·    · |    ·    ·    ·    ·  762    ·
-448    ·    ·    ·    ·    · |    ·    ·    ·    ·    ·  762
   · -288    ·    ·    ·    · |  678    ·    ·    ·    ·    ·
   · -256    ·    ·    ·    · |    ·  678    ·    ·    ·    ·
   ·  -40    ·    ·    ·    · |    ·    ·    ·  678    ·    ·
   · -192    ·    ·    ·    · |    ·    ·    ·    ·  678    ·
   · -448    ·    ·    ·    · |    ·    ·    ·    ·    ·  678
   ·    · -288    ·    ·    · |  396    ·    ·    ·    ·    ·
   ·    · -256    ·    ·    · |    ·  396    ·    ·    ·    ·
   ·    · -624    ·    ·    · |    ·    ·  396    ·    ·    ·
   ·    ·  -40    ·    ·    · |    ·    ·    ·  396    ·    ·
   ·    · -192    ·    ·    · |    ·    ·    ·    ·  396    ·
   ·    · -448    ·    ·    · |    ·    ·    ·    ·    ·  396
   ·    

In [8]:
G_b = G[[3, 8]]
G_b

array([[ -40,    0,    0,    0,    0,    0,    0,    0,    0,  762,    0,
           0],
       [   0, -624,    0,    0,    0,    0,    0,    0,  678,    0,    0,
           0]])

In [9]:
from z3 import *

def solve_with_z3(cond_a, cond_b, cond_c):
    """
    P: 法
    L: ベクトルの次元 (b_0, ..., b_{L-1})
    M_comm: 可換性条件の係数行列 (リストのリスト)
    forbidden_constraints: (係数ベクトル w, 法 g) のリスト。 w・b % g != 0 を満たす必要がある。
    """
    solver = Solver()
    
    # 変数の定義 (0 <= b_i < P)
    b = [Int(f'b_{i}') for i in range(L)]
    for x in b:
        solver.add(x >= 0, x < P)

    # 1. 等式制約: M_comm * b == 0 (mod P)
    for row in cond_a:
        # sum(row[i] * b[i]) % P == 0
        expr = Sum([row[i] * b[i] for i in range(L)])
        solver.add(expr % P == 0)

    for row in cond_b:
        expr = Sum([row[j] * b[j] for j in range(L)])
        solver.add(expr % P != 0)
        
    for row, a_c in cond_c:
        expr = Sum([row[j] * b[j] for j in range(L)])
        solver.add(expr % a_c != 0)

    # 解の探索
    if solver.check() == sat:
        model = solver.model()
        return [model[x].as_long() for x in b]
    else:
        return None

In [10]:
def is_commute(f, g):
    a_1 = f[0]
    b_1 = f[1]
    a_2 = g[0]
    b_2 = g[1]
    if ((a_1-1)*b_2 - (a_2-1)*b_1)%P == 0:
        return 1
    else:
        return 0
    

def commute_matrix(a_vec, b_vec):
    result = []
    for i in range(l_h):
        row = []
        for j in range(l_h):
            row.append(is_commute([a_vec[i], b_vec[i]], [a_vec[6+j], b_vec[6+j]]))
        result.append(row)
    return result    
        

In [ ]:
# a_vec = np.array([763, 679, 397, 61, 697, 373, 289, 257, 625, 41, 193, 449])
# b_vec = np.array([435, 69, 330, 18, 612, 246, 496, 640, 200, 524, 672, 672])
# print(np.matrix(commute_matrix(a_vec, b_vec)))

NameError: name 'b_vec' is not defined

条件Cの制約を構成する行列を作る

In [12]:
c_4 = []
for r1 in range(nrow):
    for r2 in range(r1 + 1, nrow): # r1より大きいインデックスのみ選ぶ
        for c1 in range(ncol):
            for c2 in range(c1 + 1, ncol): # c1より大きいインデックスのみ選ぶ
                pos1 = [r1, c1]
                pos2 = [r2, c2]
                c_4.append([pos1, pos2])
print(len(c_4))
c_6 = []
for r_set in itertools.combinations(list(range(nrow)), 3):
    r1, r2, r3 = r_set # 常に r1 < r2 < r3 となる
    
    # 2. 列を3つ選ぶ (組み合わせ: 12C3 = 220通り)
    for c_set in itertools.combinations(list(range(ncol)), 3):
        # 3. 3つの列の「並び順」をすべて試す (順列: 3! = 6通り)
        # これにより、固定された3行3列の中で可能な6つのサイクルをすべて生成する
        for c_perm in itertools.permutations(c_set):
            c1, c2, c3 = c_perm
            
            # 画像の定義に基づき、3つの位置をリスト化
            # [r1, c1], [r2, c2], [r3, c3]
            cycle_positions = [(r1, c1), (r2, c2), (r3, c3)]
            c_6.append(cycle_positions)
def get_valid_sequences(elements, length=4):
    
    valid = []
    for seq in product(elements, repeat=length):
        # 隣接チェック (r1!=r2, r2!=r3, r3!=r4, r4!=r1)
        if all(seq[i] != seq[(i+1)%length] for i in range(length)):
            valid.append(seq)
    return valid

def canonicalize(r_seq, c_seq):
    """サイクルを標準化して重複判定を可能にする"""
    # 4つの位置のペアを作成
    positions = tuple((r_seq[i], c_seq[i]) for i in range(4))
    
    # 8つの対称パターン（回転4種、反転4種）を生成
    symmetries = []
    curr = list(positions)
    curr_equiv = get_equiv_pos(curr)
    for _ in range(4):
        curr = curr[1:] + curr[:1]
        symmetries.append(tuple(curr))
        curr_equiv = curr_equiv[1:] + curr_equiv[:1]
        symmetries.append(tuple(curr_equiv))
    # 8つの中で辞書順最小のものを代表とする
    return min(symmetries)

def get_equiv_pos(pos):
    pos1 = tuple((pos[0][0], pos[1][1]))
    pos2 = tuple((pos[3][0], pos[0][1]))
    pos3 = tuple((pos[2][0], pos[3][1]))
    pos4 = tuple((pos[1][0], pos[2][1]))
    return [pos1, pos2, pos3, pos4]

row_seqs = get_valid_sequences(range(nrow))  # 630通り
col_seqs = get_valid_sequences(range(ncol))  # 14652通り
c_8 = []
c_8 = set()
for r in row_seqs:
    for c in col_seqs:
        c_8.add(canonicalize(r, c))
c_8 = list(c_8)

def get_cycle(pos):
    length = len(pos)
    cycle = []
    for i in range(length):
        cycle.append([pos[i][0], pos[i][1]])
        cycle.append([pos[i][0], pos[(i+1)%length][1]])
    return cycle


990


In [13]:
def composite_affine(left, right):
    a_new = (left[0] * right[0]) % P
    b_new = (left[0] * right[1] + left[1]) % P
    return [a_new, b_new]

def func_inv(input):
        try:
            a_inv = pow(input[0], -1, P)
        except ValueError:
             raise ValueError("Inverse does not exist")
        b_new = (-1 * a_inv * input[1]) % P
        return [a_inv, b_new]

In [23]:
h_x = []
for i in range(l_h):
    row = []
    for j in range(l_h):
        idx = (j-i)%l_h
        row.append(idx)
    for j in range(l_h):
        idx = (j-i)%l_h
        row.append(6+idx)
    h_x.append(row)
h_z = []
for i in range(l_h):
    row = []
    for j in range(l_h):
        idx = (i-j)%l_h
        row.append(6+idx)
    for j in range(l_h):
        idx = (i-j)%l_h
        row.append(idx)
    h_x
    h_z.append(row)
# h_x = []
# for i in range(l_h):
#     row = []
#     for j in range(l_h):
#         idx = (j-i)%l_h
#         row.append([a_vec[idx], b_vec[idx]])
#     for j in range(l_h):
#         idx = (j-i)%l_h
#         row.append([a_vec[6+idx], b_vec[6+idx]])
#     h_x.append(row)
# h_z = []
# for i in range(l_h):
#     row = []
#     for j in range(l_h):
#         idx = (i-j)%l_h
#         row.append(inv([a_vec[6+idx], b_vec[6+idx]]))
#     for j in range(l_h):
#         idx = (i-j)%l_h
#         row.append(inv([a_vec[idx], b_vec[idx]]))
# h_x

In [15]:
# def get_cycle_functions(cycle):
#     result_z = [1, 1]
#     result_z = [1, 1]
#     length = len(cycle)
#     for i in range(length // 2):
#         result_x = composite_affine(result_x, inv(h_x[cycle[2*i][0]][cycle[2*i][1]]))
#         result_x = composite_affine(result_x, h_x[cycle[2*i+1][0]][cycle[2*i+1][1]])
#         result_z = composite_affine(result_z, inv(h_z[cycle[2*i][0]][cycle[2*i][1]]))
#         result_z = composite_affine(result_z, h_z[cycle[2*i+1][0]][cycle[2*i+1][1]])
    

長さ4のサイクル

In [16]:
cycle_4 = [get_cycle(c) for c in c_4]

In [17]:
cycle = cycle_4[0]
cycle

[[0, 0], [0, 1], [1, 1], [1, 0]]

In [24]:
h_z

[[6, 11, 10, 9, 8, 7, 0, 5, 4, 3, 2, 1],
 [7, 6, 11, 10, 9, 8, 1, 0, 5, 4, 3, 2],
 [8, 7, 6, 11, 10, 9, 2, 1, 0, 5, 4, 3],
 [9, 8, 7, 6, 11, 10, 3, 2, 1, 0, 5, 4],
 [10, 9, 8, 7, 6, 11, 4, 3, 2, 1, 0, 5],
 [11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]]

In [19]:
def inv(val):
    try:
        return pow(val, -1, P)
    except ValueError:
        raise ValueError("Inverse does not exist")

In [25]:
constraints = []
cycle_4 = [get_cycle(c) for c in c_4]
for cycle in cycle_4:
    row_x = [0]*L
    row_z = [0]*L
    functions_x = []
    functions_z = []
    for i in range(4):
        functions_x.append(h_x[cycle[i][0]][cycle[i][1]])
        functions_z.append(h_z[cycle[i][0]][cycle[i][1]])
    idx_x = [functions_x[i] for i in range(4)]
    idx_z = [functions_z[i] for i in range(4)]
    a_x = [a_vec[idx] for idx in idx_x]
    a_z = [a_vec[idx] for idx in idx_z]
    
    row_x[idx_x[0]] -= inv(a_x[0])
    row_x[idx_x[1]] += inv(a_x[0])
    row_x[idx_x[2]] -= inv(a_x[0]) * a_x[1] * inv(a_x[2])
    row_x[idx_x[3]] += inv(a_x[0]) * a_x[1] * inv(a_x[2])
    a_c_x = inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] % P
    row_x = [val % P for val in row_x]
    constraints.append([row_x, a_c_x])
    
    row_z[idx_z[0]] += 1
    row_z[idx_z[1]] -= a_z[0] * inv(a_z[1])
    row_z[idx_z[2]] += a_z[0] * inv(a_z[1])
    row_z[idx_z[3]] -= a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3])
    a_c_z = a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3]) % P
    row_z = [val % P for val in row_z]
    constraints.append([row_z, a_c_z]) 

cycle_6 = [get_cycle(c) for c in c_6]
for cycle in cycle_6:
    row_x = [0]*L
    row_z = [0]*L
    functions_x = []
    functions_z = []
    for i in range(6):
        functions_x.append(h_x[cycle[i][0]][cycle[i][1]])
        functions_z.append(h_z[cycle[i][0]][cycle[i][1]])
    idx_x = [functions_x[i] for i in range(6)]
    idx_z = [functions_z[i] for i in range(6)]
    a_x = [a_vec[idx] for idx in idx_x]
    a_z = [a_vec[idx] for idx in idx_z]
    
    row_x[idx_x[0]] -= inv(a_x[0])
    row_x[idx_x[1]] += inv(a_x[0])
    row_x[idx_x[2]] -= inv(a_x[0]) * a_x[1] * inv(a_x[2])
    row_x[idx_x[3]] += inv(a_x[0]) * a_x[1] * inv(a_x[2])
    row_x[idx_x[4]] -= inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] * inv(a_x[4])
    row_x[idx_x[5]] += inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] * inv(a_x[4])
    a_c_x = inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] * inv(a_x[4]) * a_x[5] % P
    row_x = [val % P for val in row_x]
    constraints.append([row_x, a_c_x])
    
    row_z[idx_z[0]] += 1
    row_z[idx_z[1]] -= a_z[0] * inv(a_z[1])
    row_z[idx_z[2]] += a_z[0] * inv(a_z[1])
    row_z[idx_z[3]] -= a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3])
    row_z[idx_z[4]] += a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3])
    row_z[idx_z[5]] -= a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3]) * a_z[4] * inv(a_z[5])
    a_c_z = a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3]) * a_z[4] * inv(a_z[5]) % P
    row_z = [val % P for val in row_z]
    constraints.append([row_z, a_c_z])

cycle_8 = [get_cycle(c) for c in c_8]
for cycle in cycle_8:
    row_x = [0]*L
    row_z = [0]*L
    functions_x = []
    functions_z = []
    for i in range(8):
        functions_x.append(h_x[cycle[i][0]][cycle[i][1]])
        functions_z.append(h_z[cycle[i][0]][cycle[i][1]])
    idx_x = [functions_x[i] for i in range(8)]
    idx_z = [functions_z[i] for i in range(8)]
    a_x = [a_vec[idx] for idx in idx_x]
    a_z = [a_vec[idx] for idx in idx_z]
    
    row_x[idx_x[0]] -= inv(a_x[0])
    row_x[idx_x[1]] += inv(a_x[0])
    row_x[idx_x[2]] -= inv(a_x[0]) * a_x[1] * inv(a_x[2])
    row_x[idx_x[3]] += inv(a_x[0]) * a_x[1] * inv(a_x[2])
    row_x[idx_x[4]] -= inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] * inv(a_x[4])
    row_x[idx_x[5]] += inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] * inv(a_x[4])
    row_x[idx_x[6]] -= inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] * inv(a_x[4]) * a_x[5] * inv(a_x[6])
    row_x[idx_x[7]] += inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] * inv(a_x[4]) * a_x[5] * inv(a_x[6])
    a_c_x = inv(a_x[0]) * a_x[1] * inv(a_x[2]) * a_x[3] * inv(a_x[4]) * a_x[5] * inv(a_x[6]) * a_x[7] % P
    row_x = [val % P for val in row_x]
    constraints.append([row_x, a_c_x])
    
    row_z[idx_z[0]] += 1
    row_z[idx_z[1]] -= a_z[0] * inv(a_z[1])
    row_z[idx_z[2]] += a_z[0] * inv(a_z[1])
    row_z[idx_z[3]] -= a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3])
    row_z[idx_z[4]] += a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3])
    row_z[idx_z[5]] -= a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3]) * a_z[4] * inv(a_z[5])
    row_z[idx_z[6]] += a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3]) * a_z[4] * inv(a_z[5])
    row_z[idx_z[7]] -= a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3]) * a_z[4] * inv(a_z[5]) * a_z[6] * inv(a_z[7])
    a_c_z = a_z[0] * inv(a_z[1]) * a_z[2] * inv(a_z[3]) * a_z[4] * inv(a_z[5]) * a_z[6] * inv(a_z[7]) % P
    row_z = [val % P for val in row_z]
    constraints.append([row_z, a_c_z]) 


In [27]:
b_vec = solve_with_z3(G_a, G_b, constraints)